In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address      Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.0.4  105.96 KiB  16      100.0%            22870169-a5d7-46ff-a0db-a337cdcd01d3  rack1
UN  192.168.0.2  155.48 KiB  16      100.0%            249d8fc0-6ccb-4629-89a9-988d7a90f625  rack1



In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [5]:
cass.execute("drop keyspace if exists weather")
cass.execute("""create keyspace weather with replication = {
    'class' : 'SimpleStrategy',
    'replication_factor' : 3
    };
""")
cass.execute("use weather")
cass.execute("""
    create type station_record(
        tmin INT,
        tmax INT
    )
""")
cass.execute("""
    create table stations(
        id TEXT,
        name TEXT STATIC,
        date DATE,
        record WEATHER.STATION_RECORD,
        PRIMARY KEY ((id), date)
    ) WITH CLUSTERING ORDER BY (date ASC)
""")


OperationTimedOut: errors={'Connection defunct by heartbeat': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=192.168.0.2:9042

In [ ]:
#q1
cass.execute("describe table weather.stations").one().create_statement

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

In [ ]:
from pyspark.sql.functions import col, expr
import pandas as pd
df = spark.read.text("ghcnd-stations.txt")
ID_df = df.withColumn("ID", expr("substring(value, 0, 11)"))
STATE_df = df.withColumn("STATE", expr("substring(value, 39, 2)"))
NAME_df = df.withColumn("NAME", expr("substring(value, 42, 30)"))
df2 = ID_df.join(STATE_df, ['value'], how='inner')
df2 = df2.join(NAME_df, ['value'], how='inner')
filtered_df = df2.where(col("STATE") == "WI")

df_insert = cass.prepare("""
    INSERT INTO stations (id, name)
    VALUES (?, ?)
""")

filtered_df = filtered_df.toPandas()
for index, row in filtered_df.iterrows():
    cass.execute(df_insert, (filtered_df["ID"][index], filtered_df["NAME"][index]))

In [ ]:
#q2
df = pd.DataFrame(cass.execute("""
    SELECT name from weather.stations
    WHERE id='USW00014837'
"""))

df["name"][0]

In [ ]:
#q3
df = pd.DataFrame(cass.execute("""
    SELECT token(id) from weather.stations
    WHERE id='USC00470273'
"""))

df["system_token_id"][0]

In [ ]:
import subprocess
import re

In [ ]:
#q4
sp = subprocess.check_output("nodetool ring weather", stderr=subprocess.STDOUT, shell=True)
numbers = re.findall(r'-?\b\d{8,}\b', sp.decode('utf-8'))
sorted_numbers = sorted(numbers)
closest = 0
min = float('inf')
for i in range(len(sorted_numbers)):
    if (int(sorted_numbers[i]) < min):
        min = int(sorted_numbers[i])
    if (int(df["system_token_id"][0]) < int(sorted_numbers[i])):
        closest = int(sorted_numbers[i])
        break
if (closest == 0):
    closest = min
closest